In [1]:
import numpy as np
import findspark
findspark.init()
findspark.find()
import pyspark
import pyspark
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql import SparkSession
import time

In [2]:
conf = pyspark.SparkConf()
pyspark.SparkContext.setSystemProperty('spark.executor.memory', '6g')
#pyspark.SparkContext.setSystemProperty('spark.driver.memory', '14g')
sc = pyspark.SparkContext('spark://192.168.2.84:7077','Run embedding new ds')
spark = SparkSession.builder.getOrCreate()

In [3]:
RDD_4mer = sc.textFile("/kmer_counting/output_java_spark_4mer/part-00000")
positive_sequences = []

with open("/home/ubuntu/positiv_labels.txt",'r') as f:
    positive_sequences = [line.strip() for line in f.readlines()]
    
kmer_seed = 77

# randomly generated most frequent 1024 4mers from the top 3000 4mers
embedding_kmers = np.array([str(sub.split(',')[0][1:]) for sub in RDD_4mer.take(3000)])
embedding_kmers_idxs =  np.random.default_rng(seed=kmer_seed).choice(range(3000),size=1024,replace=False)
#print(embedding_kmers_idxs)
embedding_kmers = list(embedding_kmers[embedding_kmers_idxs])
#embedding_kmers[:10]

In [6]:
def encode(seq):
    bitlist = [1 if kmer in seq else 0 for kmer in embedding_kmers]
    out = []
    i=0
    for bit in bitlist:
        if i%32==0:
            out.append(0)
        out[-1] = (out[-1]<<1) | bit
        i+=1
    return out

def label_to_class(label):
    if label in positive_sequences:
        return 1
    return 0

textRDD = sc.textFile("/data/swissprot.txt")

def line_to_embedding_and_label(line):
    parts = line.split()
    label = label_to_class(parts[0])
    encoding = encode(parts[1])
    return ' '.join([str(e) for e in encoding]), ""

def toFileLine(line):
    return ' '.join(str(d) for d in line)

def apply_embedding_and_label(seed):
    seq_rdd = textRDD.map(lambda line: toFileLine(line_to_embedding_and_label(line)))
    return seq_rdd

In [8]:
start_time = time.time()
seed = 0
embedding_rdd = apply_embedding_and_label(seed)
needed_time = (time.time() - start_time)/60
embedding_rdd.saveAsTextFile("/unsupervised_embedding")
sc.stop()